<a href="https://colab.research.google.com/github/GuillermoBastian/LatamProject/blob/main/solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# --------- Predicción de demora en vuelos - LATAM --------------

# 1. Importar Librerías

In [ ]:
# Librería carga datos
from google.colab import drive 
drive.mount('/content/gdrive')

In [ ]:
#Librerías procesamiento datos y evaluación
import pandas as pd 
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
# Librerías gráficos
import seaborn as sns

In [ ]:
#Librerías Modelamiento
import lightgbm as lgbm

# 2 Lectura & Añadir Campos Challenge

## 2.1 Load data

In [ ]:
original_df = pd.read_csv('gdrive/My Drive/LATAM/dataset_SCL.csv', parse_dates=['Fecha-I', 'Fecha-O'])
feriados_chile = pd.read_excel("/content/gdrive/MyDrive/LATAM/Feriados_Chile.xlsx")
km_tpo = pd.read_excel("/content/gdrive/MyDrive/LATAM/KM_DISTANCIA.xlsx")
original_df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami


## 2.2 Borrar Variables que no agregan valor

In [ ]:

cols_borrar = ["AÑO", "Ori-O", "Ori-I", "SIGLAORI"]

def delete_cols(df, cols_borrar):
  try:
    for k in cols_borrar:
      del df[k]
    print("OK: Columnas fueron borradas")
  except:
    print("ERROR: Columnas ya habían sido borradas")
    pass  

delete_cols(original_df, cols_borrar)

OK: Columnas fueron borradas


In [ ]:
original_df.head(3)

,Fecha-I,Vlo-I,Des-I,Emp-I,Fecha-O,Vlo-O,Des-O,Emp-O,DIA,MES,DIANOM,TIPOVUELO,OPERA,SIGLADES
0,2017-01-01 23:30:00,226,KMIA,AAL,2017-01-01 23:33:00,226,KMIA,AAL,1,1,Domingo,I,American Airlines,Miami
1,2017-01-02 23:30:00,226,KMIA,AAL,2017-01-02 23:39:00,226,KMIA,AAL,2,1,Lunes,I,American Airlines,Miami
2,2017-01-03 23:30:00,226,KMIA,AAL,2017-01-03 23:39:00,226,KMIA,AAL,3,1,Martes,I,American Airlines,Miami


## 2.4 Depurar missings!

In [ ]:
original_df.isna().sum() > 0

Fecha-I      False
Vlo-I        False
Des-I        False
Emp-I        False
Fecha-O      False
Vlo-O         True
Des-O        False
Emp-O        False
DIA          False
MES          False
DIANOM       False
TIPOVUELO    False
OPERA        False
SIGLADES     False
dtype: bool

In [ ]:
#Solo faltaba un dato del Nro. de vuelo, lo completaremos con el mismo código de Santiago a Lima con LAW para día jueves
original_df["Vlo-O"] = original_df["Vlo-O"].fillna(704)

## 2.3 Agregar Variables challenge

In [ ]:
#1.- Calcular temporada_alta  -- si Fecha-I está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep, 0 si no.
original_df["temporada_alta"] = 0
original_df.loc[(original_df['Fecha-I'].dt.strftime('%m-%d') >= '12-15') | (original_df['Fecha-I'].dt.strftime('%m-%d') <= '03-03'), "temporada_alta"] = 1
original_df.loc[(original_df['Fecha-I'].dt.strftime('%m-%d') >= '07-15') & (original_df['Fecha-I'].dt.strftime('%m-%d') <= '07-31'), "temporada_alta"] = 1
original_df.loc[(original_df['Fecha-I'].dt.strftime('%m-%d') >= '09-11') & (original_df['Fecha-I'].dt.strftime('%m-%d') <= '09-30'), "temporada_alta"] = 1

#2.- Calcular dif_min 
original_df["dif_min"] = (original_df["Fecha-O"] - original_df["Fecha-I"])
original_df["dif_min"]  = original_df["dif_min"]/np.timedelta64(1,'m')

# 3.- Calcular atraso_15 
original_df["atraso_15"] = 0
original_df.loc[original_df["dif_min"] > 15, "atraso_15"] = 1

#4.- Calcular periodo_dia : mañana : 0 | tarde : 1 | noche : 2
original_df["periodo_dia"] = 0
original_df.loc[(original_df['Fecha-I'].dt.strftime('%H:%M') >= '05:00') & (original_df['Fecha-I'].dt.strftime('%H:%M') <= '11:59'), "periodo_dia"] = 0
original_df.loc[(original_df['Fecha-I'].dt.strftime('%H:%M') >= '12:00') & (original_df['Fecha-I'].dt.strftime('%H:%M') <= '18:59'), "periodo_dia"] = 1
original_df.loc[(original_df['Fecha-I'].dt.strftime('%H:%M') >= '19:00') | (original_df['Fecha-I'].dt.strftime('%H:%M') <= '04:59'), "periodo_dia"] = 2


original_df.to_csv("synthetic_features.csv")

In [ ]:
cols_borrar = ["dif_min"]
delete_cols(original_df, cols_borrar)

OK: Columnas fueron borradas


In [ ]:
original_df.head(3)

,Fecha-I,Vlo-I,Des-I,Emp-I,Fecha-O,Vlo-O,Des-O,Emp-O,DIA,MES,DIANOM,TIPOVUELO,OPERA,SIGLADES,temporada_alta,atraso_15,periodo_dia
0,2017-01-01 23:30:00,226,KMIA,AAL,2017-01-01 23:33:00,226,KMIA,AAL,1,1,Domingo,I,American Airlines,Miami,1,0,2
1,2017-01-02 23:30:00,226,KMIA,AAL,2017-01-02 23:39:00,226,KMIA,AAL,2,1,Lunes,I,American Airlines,Miami,1,0,2
2,2017-01-03 23:30:00,226,KMIA,AAL,2017-01-03 23:39:00,226,KMIA,AAL,3,1,Martes,I,American Airlines,Miami,1,0,2


## 2.4 Añadir Variables propias

In [ ]:
feriados_chile.head(3)

,Evento,Mes,Dia
0,ano_nuevo,1.0,1.0
1,viernes_santo,4.0,14.0
2,sabado_santo,4.0,15.0


In [ ]:
# 1.- Feriados:
main_nfields_df = pd.merge(original_df, feriados_chile,  how='left', left_on=['DIA','MES'], right_on = ['Dia','Mes'])
cols_borrar = ['Dia','Mes']
delete_cols(main_nfields_df, cols_borrar)
main_nfields_df['Evento'] = main_nfields_df['Evento'].fillna(0)

# 2.- Hora de partida
main_nfields_df["HORA"] = main_nfields_df["Fecha-I"].dt.hour + main_nfields_df["Fecha-I"].dt.minute/60

# 3.- Emp - Vuelo
main_nfields_df["Emp-Vlo"] = main_nfields_df["Emp-I"].astype(str) + main_nfields_df["Vlo-I"].astype(str)
main_nfields_df.head()

# 3.- Km diferencia + 4.- Tiempo de vuelo
main_nfields_df = pd.merge(main_nfields_df, km_tpo,  how='left', left_on=['SIGLADES'], right_on = ['SIGLADES'])
cols_borrar = ['Duración_hrs']
delete_cols(main_nfields_df, cols_borrar)
main_nfields_df

# 6.- último dia del mes

def calc_ult_dia (anio):
  lista_ult_dia = []
  lista_mes = []
  for i in range(1,13): 
    dummy_date = "{}-01-{}".format(str(i),str(anio))
    lista_mes.append(i)
    lista_ult_dia.append(pd.Period(dummy_date,freq='M').end_time.date().day)
    df_ult_dia = pd.DataFrame(list(zip(lista_mes, lista_ult_dia)),
               columns =['MES', 'ULT_DIA'])
  return df_ult_dia 

df_ult_dia = calc_ult_dia(2017)

main_nfields_df = pd.merge(main_nfields_df, df_ult_dia,  how='left', left_on=['MES'], right_on = ['MES'])
main_nfields_df.head()


OK: Columnas fueron borradas
OK: Columnas fueron borradas


,Fecha-I,Vlo-I,Des-I,Emp-I,Fecha-O,Vlo-O,Des-O,Emp-O,DIA,MES,...,SIGLADES,temporada_alta,atraso_15,periodo_dia,Evento,HORA,Emp-Vlo,KM,DURACION_MIN,ULT_DIA
0,2017-01-01 23:30:00,226,KMIA,AAL,2017-01-01 23:33:00,226,KMIA,AAL,1,1,...,Miami,1,0,2,ano_nuevo,23.5,AAL226,6627,497.0,31
1,2017-01-02 23:30:00,226,KMIA,AAL,2017-01-02 23:39:00,226,KMIA,AAL,2,1,...,Miami,1,0,2,0,23.5,AAL226,6627,497.0,31
2,2017-01-03 23:30:00,226,KMIA,AAL,2017-01-03 23:39:00,226,KMIA,AAL,3,1,...,Miami,1,0,2,0,23.5,AAL226,6627,497.0,31
3,2017-01-04 23:30:00,226,KMIA,AAL,2017-01-04 23:33:00,226,KMIA,AAL,4,1,...,Miami,1,0,2,0,23.5,AAL226,6627,497.0,31
4,2017-01-05 23:30:00,226,KMIA,AAL,2017-01-05 23:28:00,226,KMIA,AAL,5,1,...,Miami,1,0,2,0,23.5,AAL226,6627,497.0,31


In [ ]:
main_nfields_df.columns

Index(['Vlo-I', 'Des-I', 'Emp-I', 'Vlo-O', 'Des-O', 'Emp-O', 'DIA', 'MES',
       'DIANOM', 'TIPOVUELO', 'OPERA', 'SIGLADES', 'temporada_alta',
       'atraso_15', 'periodo_dia', 'Evento', 'HORA', 'Emp-Vlo', 'KM',
       'DURACION_MIN', 'ULT_DIA'],
      dtype='object')

In [ ]:
#Borrar últiomas columnas 
cols_borrar = ["Fecha-I", "Fecha-O", "Vlo-O", "Emp-O"]
delete_cols(main_nfields_df, cols_borrar)

ERROR: Columnas ya habían sido borradas


# 3 Análisis de datos

In [ ]:
#Reconocer si la columna es categorica o numérica
def reconocer_cate_num(df, total_cols):
  numerical_columns = []
  categorical_columns = []

  for i in total_cols:
    if df[i].dtype == object:
      categorical_columns.append(i)
    else:
      numerical_columns.append(i)
  return categorical_columns, numerical_columns

total_cols = main_nfields_df.columns
categorical_columns, numerical_columns = reconocer_cate_num(main_nfields_df, total_cols)

main_nfields_df[categorical_columns] = main_nfields_df[categorical_columns].astype(str)

In [ ]:
analysis_df = main_nfields_df

In [ ]:
# Label Encoder
# from sklearn.preprocessing import OrdinalEncoder
# oe = OrdinalEncoder()
# categorical_df = oe.fit_transform(analysis_df[categorical_columns])
# analysis_df[categorical_columns] = categorical_df

In [ ]:
# Reconocer variables independientes del target
ycols = ["atraso_15"]

def get_complement_columns(total_cols, excluded_cols):
  xcols= []
  for i in total_cols:
    if i not in ycols:
      xcols.append(i)
  return xcols

xcols = get_complement_columns(total_cols, ycols)

In [ ]:
analysis_df.describe()

,DIA,MES,temporada_alta,atraso_15,periodo_dia,HORA,KM,DURACION_MIN,ULT_DIA
count,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000
mean,15.714790,6.622585,0.333724,0.184940,0.882650,12.899395,2179.883940,181.264757,30.437381
std,8.782886,3.523321,0.471546,0.388252,0.782485,5.805976,2341.414645,163.920700,0.857487
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,301.000000,43.000000,28.000000
25%,8.000000,3.000000,0.000000,0.000000,0.000000,8.000000,916.000000,94.000000,30.000000
50%,16.000000,7.000000,0.000000,0.000000,1.000000,13.000000,1221.000000,116.000000,31.000000
75%,23.000000,10.000000,1.000000,0.000000,2.000000,17.766667,2453.000000,203.000000,31.000000
max,31.000000,12.000000,1.000000,1.000000,2.000000,23.983333,11903.000000,894.000000,31.000000


In [ ]:
#analysis_df[xcols].corr()["dif_min"]

#Resultados - VARIABLES:
# MES
# TIPOVUELO
# Vlo-I / Vlo-O
# SIGLADES

In [ ]:
analysis_df

,Vlo-I,Des-I,Emp-I,Des-O,DIA,MES,DIANOM,TIPOVUELO,OPERA,SIGLADES,temporada_alta,atraso_15,periodo_dia,Evento,HORA,Emp-Vlo,KM,DURACION_MIN,ULT_DIA
0,226,KMIA,AAL,KMIA,1,1,Domingo,I,American Airlines,Miami,1,0,2,ano_nuevo,23.500000,AAL226,6627,497.0,31
1,226,KMIA,AAL,KMIA,2,1,Lunes,I,American Airlines,Miami,1,0,2,0,23.500000,AAL226,6627,497.0,31
2,226,KMIA,AAL,KMIA,3,1,Martes,I,American Airlines,Miami,1,0,2,0,23.500000,AAL226,6627,497.0,31
3,226,KMIA,AAL,KMIA,4,1,Miercoles,I,American Airlines,Miami,1,0,2,0,23.500000,AAL226,6627,497.0,31
4,226,KMIA,AAL,KMIA,5,1,Jueves,I,American Airlines,Miami,1,0,2,0,23.500000,AAL226,6627,497.0,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,400,SPJC,JAT,SPJC,22,12,Viernes,I,JetSmart SPA,Lima,1,1,1,0,14.916667,JAT400,2453,203.0,31
68202,400,SPJC,JAT,SPJC,25,12,Lunes,I,JetSmart SPA,Lima,1,1,1,navidad,14.916667,JAT400,2453,203.0,31
68203,400,SPJC,JAT,SPJC,27,12,Miercoles,I,JetSmart SPA,Lima,1,1,1,0,14.916667,JAT400,2453,203.0,31
68204,400,SPJC,JAT,SPJC,29,12,Viernes,I,JetSmart SPA,Lima,1,0,1,0,14.916667,JAT400,2453,203.0,31


In [ ]:
# flights = main_nfields_df.pivot("MES", "Vlo-I" , "dif_min")
# ax = sns.heatmap(flights)

In [ ]:
print(len(main_nfields_df.columns))
print(len(categorical_columns))
print(len(numerical_columns))

19
10
9


# 5. Pre-Procesamiento:  
5.1 One Hot - Variables Categoricas <br>
5.2 Normalizacion - Variables Numéricas <br>

# 5.0 Label Encoding

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
le_data = main_nfields_df[categorical_columns]
le_df = main_nfields_df[categorical_columns]

le_df = main_nfields_df[categorical_columns]
oe = OrdinalEncoder()

In [ ]:
le_df[categorical_columns] = oe.fit_transform(le_data)
le_df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


,Vlo-I,Des-I,Emp-I,Des-O,DIANOM,TIPOVUELO,OPERA,SIGLADES,Evento,Emp-Vlo
0,219.0,9.0,0.0,10.0,0.0,0.0,5.0,34.0,1.0,0.0
1,219.0,9.0,0.0,10.0,2.0,0.0,5.0,34.0,0.0,0.0
2,219.0,9.0,0.0,10.0,3.0,0.0,5.0,34.0,0.0,0.0
3,219.0,9.0,0.0,10.0,4.0,0.0,5.0,34.0,0.0,0.0
4,219.0,9.0,0.0,10.0,1.0,0.0,5.0,34.0,0.0,0.0


## 5.1 One - Hot: Categoricas

In [ ]:
one_hot_status= 'N'

In [ ]:

if one_hot_status == 'N':
  for i in categorical_columns:
    main_onehot_df = pd.get_dummies(main_nfields_df[i], prefix=i)
    one_hot_status= 'Y'


main_onehot_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68206 entries, 0 to 68205
Columns: 685 entries, Emp-Vlo_AAL226 to Emp-Vlo_UAL846A
dtypes: uint8(685)
memory usage: 45.1 MB


# 5.2 Normalizar variables numéricas

In [131]:
def normalizar_df(df):
  scaler = StandardScaler()
  scaler.fit(df)
  scaled = scaler.fit_transform(df)
  scaled_df = pd.DataFrame(scaled, columns=df.columns)
  return scaled_df

In [132]:
norm_df = normalizar_df(main_nfields_df[numerical_columns])*100
norm_df.head()

,DIA,MES,temporada_alta,atraso_15,periodo_dia,HORA,KM,DURACION_MIN,ULT_DIA
0,-167.540592,-159.583128,141.297041,-47.634352,142.796198,182.582277,189.934264,192.616038,65.613028
1,-156.154728,-159.583128,141.297041,-47.634352,142.796198,182.582277,189.934264,192.616038,65.613028
2,-144.768865,-159.583128,141.297041,-47.634352,142.796198,182.582277,189.934264,192.616038,65.613028
3,-133.383002,-159.583128,141.297041,-47.634352,142.796198,182.582277,189.934264,192.616038,65.613028
4,-121.997139,-159.583128,141.297041,-47.634352,142.796198,182.582277,189.934264,192.616038,65.613028


# 6. Train - Test Split

## 6.1 Get input scenarios


In [ ]:
input_model = pd.concat([le_df , main_nfields_df[numerical_columns]], axis=1) #Escenario 1: Normal - Label encoder + variables numéricas
input_model_onehot = pd.concat([main_onehot_df, main_nfields_df[numerical_columns]], axis=1) #Escenario 2: onehot - OneHot + variables numéricas
input_model_norm = pd.concat([le_df[categorical_columns], norm_df] , axis=1) #Escenario 3: Normalizadas - Label encoder + Numerico_Normalizado
input_model_onehot_norm = pd.concat([main_onehot_df, norm_df] , axis=1) #Escenario 4: All - Onehot - Label Numerico_Normalizado 


input_model_onehot["atraso_15"] = input_model["atraso_15"]
input_model_norm["atraso_15"] = input_model["atraso_15"]
input_model_onehot_norm["atraso_15"] = input_model["atraso_15"]

## 6.2 Get X and Y

In [ ]:
# Reconocer variables independientes del target
ycols = ["atraso_15", "dif_min"]
y_cols_clasiff = ["atraso_15"]

xcols_input = get_complement_columns(input_model.columns, ycols)
xcols_onehot = get_complement_columns(input_model_onehot.columns, ycols)
xcols_norm = get_complement_columns(input_model_norm.columns, ycols)
xcols_onehot_norm = get_complement_columns(input_model_onehot_norm.columns, ycols)


## 6.1. Oversampling

In [98]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE 
import imblearn
from numpy import where
from matplotlib import pyplot

In [73]:
input_model.shape[0]

68206

In [84]:
input_model["atraso_15"].sum()
#0: 55592
#1: 12614
#total : 68206

12614

In [116]:
oversample = SMOTE()
X_sm, y_sm = oversample.fit_resample(input_model[xcols_input], input_model[y_cols_clasiff])

In [117]:
X_y_sm = pd.concat([X_sm, y_sm], axis=1)

In [133]:
xcols_norm_sm = get_complement_columns(X_y_sm.columns, ycols)

## 6.2 Train - Test Split

In [113]:
ts = 0.15 #test size
strtfy_by = ["atraso_15", "MES", "TIPOVUELO"] 

X_train_sm, X_test_sm, y_train_sm, y_test_sm = train_test_split(X_sm, y_sm, test_size=ts, random_state=42, stratify = X_y_sm["atraso_15"], shuffle=True)
X_train, X_test, y_train, y_test = train_test_split(input_model[xcols_input], input_model[y_cols_clasiff], test_size=ts, random_state=42, stratify = input_model[strtfy_by], shuffle=True)
X_train_oh, X_test_oh, y_train_oh, y_test_oh = train_test_split(input_model_onehot[xcols_onehot], input_model_onehot[y_cols_clasiff], test_size=ts, random_state=42, stratify = input_model_onehot[y_cols_clasiff], shuffle=True)
X_train_nor, X_test_nor, y_train_nor, y_test_nor = train_test_split(input_model_norm[xcols_norm], input_model_norm[y_cols_clasiff], test_size=ts, random_state=42, stratify = input_model_norm[y_cols_clasiff], shuffle=True)
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(input_model_onehot_norm[xcols_onehot_norm], input_model_onehot_norm[y_cols_clasiff], test_size=ts, random_state=42, stratify = input_model_onehot_norm[y_cols_clasiff], shuffle=True)


In [ ]:
a,b = y_train.value_counts(normalize = True).tolist()  
oversampling_rate = a/b
oversampling_rate

4.407611230295681

In [ ]:
y_train.loc[y_train.atraso_15 == 1]

,atraso_15
22851,1
30018,1
13078,1
37842,1
36708,1
...,...
38058,1
19778,1
6205,1
61821,1


# 7. Modeling

In [ ]:
# class_weight = {0: (number of negative samples / number of positive samples), 
#                 1: (number of positive samples / number of negative samples)}

In [ ]:
opt_depth = 6
opt_leaves = 2**opt_depth
 

clf = lgbm.LGBMClassifier(n_estimators=5000, scale_pos_weight = 5.5,  num_leaves=opt_leaves, max_depth = opt_depth, learning_rate=0.01)
#boosting_type='gbdt', learning_rate=0.1, n_estimators=5000,
#                          n_jobs=-1, random_state=42, scale_pos_weight = 5, num_leaves=opt_leaves, max_depth = opt_depth
clf.fit(X_train, y_train, eval_set= [(X_test, y_test)], eval_metric = 'auc')

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1]	valid_0's binary_logloss: 0.47798	valid_0's auc: 0.674518
[2]	valid_0's binary_logloss: 0.477142	valid_0's auc: 0.677179
[3]	valid_0's binary_logloss: 0.476454	valid_0's auc: 0.67818
[4]	valid_0's binary_logloss: 0.475826	valid_0's auc: 0.67954
[5]	valid_0's binary_logloss: 0.475322	valid_0's auc: 0.679775
[6]	valid_0's binary_logloss: 0.474916	valid_0's auc: 0.679572
[7]	valid_0's binary_logloss: 0.474565	valid_0's auc: 0.680608
[8]	valid_0's binary_logloss: 0.474318	valid_0's auc: 0.681947
[9]	valid_0's binary_logloss: 0.474153	valid_0's auc: 0.683256
[10]	valid_0's binary_logloss: 0.474088	valid_0's auc: 0.683539
[11]	valid_0's binary_logloss: 0.474041	valid_0's auc: 0.684963
[12]	valid_0's binary_logloss: 0.474076	valid_0's auc: 0.68491
[13]	valid_0's binary_logloss: 0.47418	valid_0's auc: 0.685215
[14]	valid_0's binary_logloss: 0.474349	valid_0's auc: 0.685657
[15]	valid_0's binary_logloss: 0.474593	valid_0's auc: 0.687763
[16]	valid_0's binary_logloss: 0.474841	valid_0's auc:

LGBMClassifier(learning_rate=0.01, max_depth=6, n_estimators=5000,
               num_leaves=64, scale_pos_weight=5.5)

In [ ]:
clf_best = clf.best_iteration_

In [ ]:
clf_oh = lgbm.LGBMClassifier(boosting_type='gbdt', class_weight=None, importance_type='split', learning_rate=0.005, n_estimators=5000,
                          n_jobs=-1, num_leaves=31, random_state=42, scale_pos_weight = 4.5)
clf_oh.fit(X_train_oh, y_train_oh)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMClassifier(learning_rate=0.005, n_estimators=5000, random_state=42,
               scale_pos_weight=4.5)

In [ ]:
opt_depth = 12
opt_leaves = 2**opt_depth
 
clf_nor = lgbm.LGBMClassifier(boosting_type='gbdt', learning_rate=0.015, n_estimators=5000,
                          n_jobs=-1, random_state=42, num_leaves=opt_leaves, max_depth = opt_depth, is_unbalance= True)
clf_nor.fit(X_train_nor, y_train_oh, eval_set= [(X_test_nor, y_test_oh)], eval_metric = 'auc')

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1]	valid_0's binary_logloss: 0.476387	valid_0's auc: 0.688503
[2]	valid_0's binary_logloss: 0.474219	valid_0's auc: 0.691862
[3]	valid_0's binary_logloss: 0.472335	valid_0's auc: 0.693668
[4]	valid_0's binary_logloss: 0.470578	valid_0's auc: 0.69831
[5]	valid_0's binary_logloss: 0.46903	valid_0's auc: 0.701059
[6]	valid_0's binary_logloss: 0.467651	valid_0's auc: 0.702782
[7]	valid_0's binary_logloss: 0.46648	valid_0's auc: 0.70376
[8]	valid_0's binary_logloss: 0.465522	valid_0's auc: 0.703595
[9]	valid_0's binary_logloss: 0.464571	valid_0's auc: 0.705261
[10]	valid_0's binary_logloss: 0.463833	valid_0's auc: 0.70552
[11]	valid_0's binary_logloss: 0.463134	valid_0's auc: 0.70643
[12]	valid_0's binary_logloss: 0.462623	valid_0's auc: 0.70649
[13]	valid_0's binary_logloss: 0.462125	valid_0's auc: 0.707192
[14]	valid_0's binary_logloss: 0.46173	valid_0's auc: 0.708146
[15]	valid_0's binary_logloss: 0.461445	valid_0's auc: 0.708458
[16]	valid_0's binary_logloss: 0.46124	valid_0's auc: 0.7

KeyboardInterrupt: ignored

In [ ]:
opt_depth = 12
opt_leaves = 2**opt_depth
 

clf_oh_nor = lgbm.LGBMClassifier(boosting_type='gbdt', learning_rate=0.015, n_estimators=5000,
                          n_jobs=-1, random_state=42, scale_pos_weight = 5, num_leaves=opt_leaves, max_depth = opt_depth)
clf_oh_nor.fit(X_train_all, y_train_oh, eval_set= [(X_test_all, y_test_oh)], eval_metric = 'auc')

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1]	valid_0's binary_logloss: 0.47702	valid_0's auc: 0.674334
[2]	valid_0's binary_logloss: 0.475385	valid_0's auc: 0.685804
[3]	valid_0's binary_logloss: 0.473976	valid_0's auc: 0.688563
[4]	valid_0's binary_logloss: 0.47283	valid_0's auc: 0.689713
[5]	valid_0's binary_logloss: 0.471786	valid_0's auc: 0.696683
[6]	valid_0's binary_logloss: 0.470851	valid_0's auc: 0.700307
[7]	valid_0's binary_logloss: 0.470144	valid_0's auc: 0.701285
[8]	valid_0's binary_logloss: 0.469652	valid_0's auc: 0.701484
[9]	valid_0's binary_logloss: 0.469354	valid_0's auc: 0.703365
[10]	valid_0's binary_logloss: 0.469038	valid_0's auc: 0.704096
[11]	valid_0's binary_logloss: 0.468814	valid_0's auc: 0.704725
[12]	valid_0's binary_logloss: 0.468703	valid_0's auc: 0.706403
[13]	valid_0's binary_logloss: 0.468756	valid_0's auc: 0.707009
[14]	valid_0's binary_logloss: 0.468869	valid_0's auc: 0.707663
[15]	valid_0's binary_logloss: 0.469082	valid_0's auc: 0.707952
[16]	valid_0's binary_logloss: 0.469294	valid_0's a

KeyboardInterrupt: ignored

In [157]:
opt_depth = 7
opt_leaves = 2**opt_depth
 

# clf_oh_nor = lgbm.LGBMClassifier(boosting_type='gbdt', learning_rate=0.015, n_estimators=5000,
#                           n_jobs=-1, random_state=42, scale_pos_weight = 5, num_leaves=opt_leaves, max_depth = opt_depth)
# clf_oh_nor.fit(X_train_sm, y_train_sm, eval_set= [(X_test_sm, y_test_sm)], eval_metric = 'auc')

# clf_sm = lgbm.LGBMClassifier(boosting_type='gbdt', learning_rate=0.001, n_estimators=5000,
#                            n_jobs=-1, random_state=42, scale_pos_weight = 1, num_leaves=opt_leaves, max_depth = opt_depth)
# clf_sm.fit(X_train_sm, y_train_sm, eval_set= [(X_test_sm, y_test_sm)], eval_metric = 'auc')

# LightGBM OneHot Model accuracy score: 0.8385
# LightGBM OneHot Model accuracy score: 0.8324
# LightGBM OneHot Model accuracy score: 0.8503

clf_sm = lgbm.LGBMClassifier(boosting_type='gbdt', learning_rate=0.005, n_estimators=5000,
                           n_jobs=-1, random_state=42, scale_pos_weight = 1, num_leaves=opt_leaves, max_depth = opt_depth)
clf_sm.fit(X_train_sm, y_train_sm, eval_set= [(X_test_sm, y_test_sm)], eval_metric = 'auc')

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1]	valid_0's binary_logloss: 0.691884	valid_0's auc: 0.76163
[2]	valid_0's binary_logloss: 0.690634	valid_0's auc: 0.761635
[3]	valid_0's binary_logloss: 0.689395	valid_0's auc: 0.761636
[4]	valid_0's binary_logloss: 0.68817	valid_0's auc: 0.761635
[5]	valid_0's binary_logloss: 0.686956	valid_0's auc: 0.761635
[6]	valid_0's binary_logloss: 0.685754	valid_0's auc: 0.76163
[7]	valid_0's binary_logloss: 0.684564	valid_0's auc: 0.761693
[8]	valid_0's binary_logloss: 0.683387	valid_0's auc: 0.761743
[9]	valid_0's binary_logloss: 0.682221	valid_0's auc: 0.761714
[10]	valid_0's binary_logloss: 0.681067	valid_0's auc: 0.761673
[11]	valid_0's binary_logloss: 0.679923	valid_0's auc: 0.761699
[12]	valid_0's binary_logloss: 0.678792	valid_0's auc: 0.761645
[13]	valid_0's binary_logloss: 0.677668	valid_0's auc: 0.761661
[14]	valid_0's binary_logloss: 0.67656	valid_0's auc: 0.761662
[15]	valid_0's binary_logloss: 0.675459	valid_0's auc: 0.761655
[16]	valid_0's binary_logloss: 0.67437	valid_0's auc:

LGBMClassifier(learning_rate=0.005, max_depth=7, n_estimators=5000,
               num_leaves=128, random_state=42, scale_pos_weight=1)

In [158]:
#Y_pred = clf.predict(X_test, num_iteration=clf.best_iteration_)
#Y_pred
# Y_pred_oh = clf_oh.predict(X_test_oh)
# Y_pred_nor = clf_nor.predict(X_test_nor)
# Y_pred_all = clf_oh_nor.predict(X_test_all)
y_pred_train_sm = clf_sm.predict(X_train_sm,  num_iteration=clf_sm.best_iteration_)
y_pred_sm = clf_sm.predict(X_test_sm,  num_iteration=clf_sm.best_iteration_)

In [159]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
#f1_oh=f1_score(Y_pred_oh, y_test_oh)
#f1_nor=f1_score(Y_pred_nor, y_test_nor)
#f1_all=f1_score(Y_pred_all, y_test_all)

print('LightGBM OneHot Model accuracy score: {0:0.4f}'.format(f1_score(y_train_sm, y_pred_train_sm)))
print('LightGBM OneHot Model accuracy score: {0:0.4f}'.format(f1_score(y_test_sm, y_pred_sm)))
print('LightGBM OneHot Model accuracy score: {0:0.4f}'.format(roc_auc_score(y_test_sm, y_pred_sm)))
# print('LightGBM OneHot Model accuracy score: {0:0.4f}'.format(f1_score(y_test, Y_pred_oh)))
# print('LightGBM Normalization Model accuracy score: {0:0.4f}'.format(f1_score(y_test, Y_pred_nor)))
# print('LightGBM OneHot + Normalization Model accuracy score: {0:0.4f}'.format(f1_score(y_test, Y_pred_all)))

LightGBM OneHot Model accuracy score: 0.8848
LightGBM OneHot Model accuracy score: 0.8645
LightGBM OneHot Model accuracy score: 0.8753


# 7. Modeling with Deep Learning

In [ ]:
import tensorflow as tf # NN, LOSS FUNCTION, METRICS, ETC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import * 
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
es_cl = EarlyStopping(monitor = 'loss', mode='max', patience = 30, verbose = 1)
mc_cb = ModelCheckpoint(filepath = "bestmodel1.hdf5", monitor= 'loss', save_best_only= True)
#.BinaryAccuracy()

In [ ]:
tf.random.set_seed(40) #SEMILA ALEATORIA PARA QUE LA SUERTE NO SEA DETERMINANTE NUNCA


model = Sequential()
model.add(Dense(512,activation='relu', input_shape = (693,)))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=tf.keras.metrics.BinaryAccuracy())



In [ ]:
X_train_all.shape[1]

693

In [ ]:
history = model.fit(X_train_all, y_train_oh, validation_data = (X_test_all,y_test_oh), epochs = 100, callbacks = [es_cl, mc_cb], batch_size=128)


Epoch 1/100
453/453 [==============================] - 9s 20ms/step - loss: 0.3747 - binary_accuracy: 0.8407 - val_loss: 0.4717 - val_binary_accuracy: 0.8117
Epoch 2/100
453/453 [==============================] - 7s 14ms/step - loss: 0.3748 - binary_accuracy: 0.8398 - val_loss: 0.4630 - val_binary_accuracy: 0.8144
Epoch 3/100
453/453 [==============================] - 6s 14ms/step - loss: 0.3728 - binary_accuracy: 0.8405 - val_loss: 0.4674 - val_binary_accuracy: 0.8117
Epoch 4/100
453/453 [==============================] - 8s 17ms/step - loss: 0.3732 - binary_accuracy: 0.8404 - val_loss: 0.4648 - val_binary_accuracy: 0.8168
Epoch 5/100
453/453 [==============================] - 8s 17ms/step - loss: 0.3719 - binary_accuracy: 0.8397 - val_loss: 0.4752 - val_binary_accuracy: 0.8114
Epoch 6/100
 58/453 [==>...........................] - ETA: 4s - loss: 0.3740 - binary_accuracy: 0.8429

KeyboardInterrupt: ignored